In [46]:
import pandas as pd
import itertools

In [47]:
df = pd.read_csv('../data/2_merged_da_data.csv')
impact_preds = pd.read_csv('../data/1_impact_predictions.csv')

driver_df = pd.read_csv('../data/1_driver_predictions.csv')
relevance_preds = pd.read_csv('../data/1_document_relevance.csv')
impact_drivers = impact_preds.merge(driver_df).merge(relevance_preds)

doc_continents = pd.read_csv('../data/doc_continents.csv').merge(pd.read_csv('../data/1_document_relevance.csv'), left_on="doc_id",right_on="id")

In [48]:
shp_ndf_df = pd.read_csv('../data/2_country_data.csv')

In [49]:
df_ndf = pd.read_csv(f'../data/study_gridcell_2.5.csv')
df_ndf = df_ndf.merge(df_ndf.groupby('doc_id').count().reset_index().rename(columns={"ndf_id":"gridcells"}))
df_ndf['weight'] = 1/df_ndf['gridcells']
df_ndf.head()

,ndf_id,doc_id,gridcells,weight
0,7674.0,3323227.0,6,0.166667
1,6804.0,3323227.0,6,0.166667
2,6720.0,3323227.0,6,0.166667
3,7327.0,3323227.0,6,0.166667
4,7674.0,3323227.0,6,0.166667


In [50]:
sector_cols = [x for x in impact_drivers.columns if "12 - " in x and "mean" in x]

continents = [
    "South America", 
    "North America", 
    "Africa", 
    "Europe", 
    "Asia",
    "Oceania"
]

sector_cols

['12 - Coastal and marine Ecosystems - mean_prediction',
 '12 - Human and managed - mean_prediction',
 '12 - Mountains, snow and ice - mean_prediction',
 '12 - Rivers, lakes, and soil moisture - mean_prediction',
 '12 - Terrestrial ES - mean_prediction',
 '12 - Other systems - mean_prediction']

In [51]:
sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]
sector_labels = [x + " (WS>5)" if "Total" in x else x + " (WS>1)" for x in sector_labels ]

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[sector_labels,["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[sub_ndf[f"total_study_prop{suffix}"]>=threshold][var].sum()/ndf_sum)
        
    return f"{results[0]:.0%} ({results[1]:.0%}-{results[2]:.0%})"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(continents+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        ndf_ids = shp_ndf_df[shp_ndf_df['CONTINENT']==continent].ndf_id
        group = doc_continents[doc_continents["continent"]==continent] 
        cdf = impact_drivers[impact_drivers['id'].isin(group.doc_id)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
            thresh = 5
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
            thresh = 1
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector_labels[k], var)] = ci_area_sum(ndf, sub_ndf,thresh, var)       

table.to_html('../supplementary_data/area_pop_sums.md')
table.to_excel('../supplementary_data/area_pop_sums.xlsx')
                
table

Coastal and marine Ecosystems (WS>1)                 \
                                                    area     population   
South America D&A                            8% (5%-31%)  26% (18%-51%)   
              Other                          3% (1%-27%)    6% (2%-15%)   
North America D&A                          33% (19%-50%)  58% (43%-70%)   
              Other                         11% (5%-12%)   27% (9%-27%)   
Africa        D&A                           12% (4%-23%)  22% (12%-37%)   
              Other                           2% (0%-4%)     1% (0%-4%)   
Europe        D&A                          16% (11%-21%)  59% (41%-71%)   
              Other                           0% (0%-0%)     0% (0%-0%)   
Asia          D&A                           19% (6%-29%)  47% (18%-58%)   
              Other                           3% (1%-4%)    9% (4%-13%)   
Oceania       D&A                          76% (72%-76%)  91% (59%-91%)   
              Other                        14% (14%-14%)     5% (5%-5%)   
Global        D&A                          22% (13%-33%)  44% (23%-57%)   
              Other                           5% (2%-8%)    8% (3%-11%)   

                    Human and managed systems (WS>1)                 \
                                                area     population   
South America D&A                      26% (13%-62%)  55% (38%-79%)   
              Other                     24% (5%-34%)   14% (9%-17%)   
North America D&A                      40% (25%-57%)  64% (48%-72%)   
              Other                      9% (7%-15%)  28% (26%-28%)   
Africa        D&A                      32% (25%-46%)  68% (58%-79%)   
              Other                      7% (3%-14%)     3% (3%-5%)   
Europe        D&A                      25% (20%-33%)  80% (73%-88%)   
              Other                       1% (0%-1%)     0% (0%-0%)   
Asia          D&A                      61% (44%-78%)  80% (75%-82%)   
              Other                       5% (5%-7%)  15% (15%-15%)   
Oceania       D&A                      74% (53%-84%)  70% (54%-86%)   
              Other                    14% (14%-14%)     5% (5%-5%)   
Global        D&A                      38% (27%-53%)  73% (64%-79%)   
              Other                      8% (4%-12%)  11% (11%-12%)   

                    Mountains, snow and ice (WS>1)                 \
                                              area     population   
South America D&A                     11% (8%-15%)  19% (14%-32%)   
              Other                   13% (6%-14%)    6% (3%-10%)   
North America D&A                    34% (18%-54%)  43% (27%-56%)   
              Other                    8% (5%-19%)   19% (8%-24%)   
Africa        D&A                       3% (1%-9%)    7% (6%-14%)   
              Other                     0% (0%-1%)     0% (0%-1%)   
Europe        D&A                     21% (8%-30%)  58% (25%-79%)   
              Other                     0% (0%-1%)     0% (0%-0%)   
Asia          D&A                    27% (14%-43%)  39% (12%-56%)   
              Other                     4% (2%-4%)   14% (6%-14%)   
Oceania       D&A                       3% (2%-8%)  37% (15%-53%)   
              Other                     1% (1%-3%)     4% (4%-4%)   
Global        D&A                     17% (9%-27%)  32% (13%-46%)   
              Other                     4% (2%-7%)    9% (4%-10%)   

                    Rivers, lakes, and soil moisture (WS>1)                 \
                                                       area     population   
South America D&A                              16% (6%-45%)  33% (14%-68%)   
              Other                            19% (3%-34%)   15% (5%-19%)   
North America D&A                             52% (24%-64%)  60% (41%-69%)   
              Other                            15% (9%-18%)  28% (28%-28%)   
Africa        D&A                              17% (5%-35%)  44% (18%-68%)   
              Other            

In [52]:
ltable = table.copy()
for c in table.columns:
    ltable[c] = ltable[c].str.replace(' (',' \mbox{(',regex=False).str.replace(')',')}').str.replace('%','\%')
ltable.index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global"],["D\&A","Other"]])))
ltable.to_latex('../tables/area_pop_sums.tex',escape=False,column_format="ll " + " ".join(["p{1cm}"]*14))
ltable.to_latex('../tables/area_pop_sums.tex',escape=False,multicolumn_format="L{2cm}",column_format="ll " + " ".join(["p{1cm}"]*14))

In [91]:
sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]
sector_labels = [x + " (WS>5)" if "Total" in x else x + " (WS>1)" for x in sector_labels ]

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[sector_labels,["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[sub_ndf[f"total_study_prop{suffix}"]>=threshold][var].sum()/ndf_sum)
        
    return f"{results[0]:.0%} ({results[1]:.0%}-{results[2]:.0%})"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(continents+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        ndf_ids = shp_ndf_df[shp_ndf_df['CONTINENT']==continent].ndf_id
        group = doc_continents[doc_continents["continent"]==continent] 
        cdf = impact_drivers[impact_drivers['id'].isin(group.doc_id)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
            thresh = 20
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
            thresh = 4
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector_labels[k], var)] = ci_area_sum(ndf, sub_ndf,thresh, var)       

table.to_html('../supplementary_data/area_pop_sums_high.md')
table.to_excel('../supplementary_data/area_pop_sums_high.xlsx')
                
table

Coastal and marine Ecosystems (WS>1)                       \
                                                    area           population   
South America D\&A                     8%\mbox{(5%-31%)}  26%\mbox{(18%-51%)}   
              Other                    3%\mbox{(1%-27%)}    6%\mbox{(2%-15%)}   
North America D\&A                   33%\mbox{(19%-50%)}  58%\mbox{(43%-70%)}   
              Other                   11%\mbox{(5%-12%)}   27%\mbox{(9%-27%)}   
Africa        D\&A                    12%\mbox{(4%-23%)}  22%\mbox{(12%-37%)}   
              Other                     2%\mbox{(0%-4%)}     1%\mbox{(0%-4%)}   
Europe        D\&A                   16%\mbox{(11%-21%)}  59%\mbox{(41%-71%)}   
              Other                     0%\mbox{(0%-0%)}     0%\mbox{(0%-0%)}   
Asia          D\&A                    19%\mbox{(6%-29%)}  47%\mbox{(18%-58%)}   
              Other                     3%\mbox{(1%-4%)}    9%\mbox{(4%-13%)}   
Oceania       D\&A                   76%\mbox{(72%-76%)}  91%\mbox{(59%-91%)}   
              Other                  14%\mbox{(14%-14%)}     5%\mbox{(5%-5%)}   
Global        D\&A                   22%\mbox{(13%-33%)}  44%\mbox{(23%-57%)}   
              Other                     5%\mbox{(2%-8%)}    8%\mbox{(3%-11%)}   

                    Human and managed systems (WS>1)                       \
                                                area           population   
South America D\&A               26%\mbox{(13%-62%)}  55%\mbox{(38%-79%)}   
              Other               24%\mbox{(5%-34%)}   14%\mbox{(9%-17%)}   
North America D\&A               40%\mbox{(25%-57%)}  64%\mbox{(48%-72%)}   
              Other                9%\mbox{(7%-15%)}  28%\mbox{(26%-28%)}   
Africa        D\&A               32%\mbox{(25%-46%)}  68%\mbox{(58%-79%)}   
              Other                7%\mbox{(3%-14%)}     3%\mbox{(3%-5%)}   
Europe        D\&A               25%\mbox{(20%-33%)}  80%\mbox{(73%-88%)}   
              Other                 1%\mbox{(0%-1%)}     0%\mbox{(0%-0%)}   
Asia          D\&A               61%\mbox{(44%-78%)}  80%\mbox{(75%-82%)}   
              Other                 5%\mbox{(5%-7%)}  15%\mbox{(15%-15%)}   
Oceania       D\&A               74%\mbox{(53%-84%)}  70%\mbox{(54%-86%)}   
              Other              14%\mbox{(14%-14%)}     5%\mbox{(5%-5%)}   
Global        D\&A               38%\mbox{(27%-53%)}  73%\mbox{(64%-79%)}   
              Other                8%\mbox{(4%-12%)}  11%\mbox{(11%-12%)}   

                    Mountains, snow and ice (WS>1)                       \
                                              area           population   
South America D\&A              11%\mbox{(8%-15%)}  19%\mbox{(14%-32%)}   
              Other             13%\mbox{(6%-14%)}    6%\mbox{(3%-10%)}   
North America D\&A             34%\mbox{(18%-54%)}  43%\mbox{(27%-56%)}   
              Other              8%\mbox{(5%-19%)}   19%\mbox{(8%-24%)}   
Africa        D\&A                3%\mbox{(1%-9%)}    7%\mbox{(6%-14%)}   
              Other               0%\mbox{(0%-1%)}     0%\mbox{(0%-1%)}   
Europe        D\&A              21%\mbox{(8%-30%)}  58%\mbox{(25%-79%)}   
              Other               0%\mbox{(0%-1%)}     0%\mbox{(0%-0%)}   
Asia          D\&A             27%\mbox{(14%-43%)}  39%\mbox{(12%-56%)}   
              Other               4%\mbox{(2%-4%)}   14%\mbox{(6%-14%)}   
Oceania       D\&A                3%\mbox{(2%-8%)}  37%\mbox{(15%-53%)}   
              Other               1%\mbox{(1%-3%)}     4%\mbox{(4%-4%)}   
Global        D\&A              17%\mbox{(9%-27%)}  32%\mbox{(13%-46%)}   
              Other               4%\mbox{(2%-7%)}    9%\mbox{(4%-10%)}   

                    Rivers, lakes, and soil moisture (WS>1)  \
                                                       area   
South America D\&A                       16%\mbox{(6%-45%)}   
              Other                      19%\mbox{(3%-34%)}   
North America D\&A                   

In [ ]:
ltable = table.copy()
for c in table.columns:
    ltable[c] = ltable[c].str.replace(' (',' \mbox{(',regex=False).str.replace(')',')}').str.replace('%','\%')
ltable.index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global"],["D\&A","Other"]])))
ltable.to_latex('../tables/area_pop_sums_high.tex',escape=False,column_format="ll " + " ".join(["p{1cm}"]*14))
ltable.to_latex('../tables/area_pop_sums_high.tex',escape=False,multicolumn_format="L{2cm}",column_format="ll " + " ".join(["p{1cm}"]*14))

In [111]:
sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]
sector_labels = [x + " (WS>5)" if "Total" in x else x + " (WS>1)" for x in sector_labels ]

incs = [x[3:] for x in shp_ndf_df['Income category'].unique()]

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[sector_labels,["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[incs + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[sub_ndf[f"total_study_prop{suffix}"]>=threshold][var].sum()/ndf_sum)
        
    return f"{results[0]:.0%} ({results[1]:.0%}-{results[2]:.0%})"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(incs+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        group = shp_ndf_df[shp_ndf_df['Income category'].str.contains(continent)]
        ndf_ids = group.ndf_id
        doc_ids = df_ndf[df_ndf['ndf_id'].isin(ndf_ids)].doc_id
        cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
            thresh = 5
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
            thresh = 1
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector_labels[k], var)] = ci_area_sum(ndf, sub_ndf,thresh, var)       

table.to_html('../supplementary_data/area_pop_sums_income.md')
table.to_excel('../supplementary_data/area_pop_sums_income.xlsx')
                
table

Coastal and marine Ecosystems (WS>1)                 \
                                                          area     population   
Low income          D&A                            8% (2%-20%)   17% (5%-27%)   
                    Other                           3% (0%-5%)  12% (11%-17%)   
Lower middle income D&A                           26% (9%-37%)  59% (19%-65%)   
                    Other                           5% (1%-8%)   10% (2%-16%)   
Upper middle income D&A                           11% (5%-23%)  29% (17%-49%)   
                    Other                           2% (1%-9%)     4% (1%-6%)   
High Income         D&A                          45% (34%-56%)  77% (60%-85%)   
                    Other                          9% (6%-10%)     9% (6%-9%)   
Global              D&A                          22% (13%-33%)  44% (23%-57%)   
                    Other                           5% (2%-8%)    8% (3%-11%)   

                          Human and managed systems (WS>1)                 \
                                                      area     population   
Low income          D&A                      35% (29%-45%)  53% (46%-61%)   
                    Other                      5% (4%-11%)  20% (20%-21%)   
Lower middle income D&A                      50% (42%-74%)  77% (71%-80%)   
                    Other                     14% (9%-19%)  18% (18%-18%)   
Upper middle income D&A                      34% (21%-51%)  80% (69%-88%)   
                    Other                      9% (3%-11%)     6% (4%-6%)   
High Income         D&A                      50% (37%-66%)  85% (80%-90%)   
                    Other                      8% (6%-13%)     9% (8%-9%)   
Global              D&A                      38% (27%-53%)  73% (64%-79%)   
                    Other                      8% (4%-12%)  11% (11%-12%)   

                          Mountains, snow and ice (WS>1)                 \
                                                    area     population   
Low income          D&A                      6% (3%-11%)    6% (3%-17%)   
                    Other                     1% (1%-2%)   16% (5%-17%)   
Lower middle income D&A                     22% (6%-29%)  49% (12%-58%)   
                    Other                   12% (3%-12%)   17% (7%-17%)   
Upper middle income D&A                     13% (9%-24%)  19% (11%-42%)   
                    Other                     2% (2%-3%)     2% (0%-4%)   
High Income         D&A                    32% (16%-48%)  68% (37%-82%)   
                    Other                    5% (3%-13%)     7% (6%-8%)   
Global              D&A                     17% (9%-27%)  32% (13%-46%)   
                    Other                     4% (2%-7%)    9% (4%-10%)   

                          Rivers, lakes, and soil moisture (WS>1)  \
                                                             area   
Low income          D&A                              22% (7%-37%)   
                    Other                              4% (2%-8%)   
Lower middle income D&A                             38% (19%-59%)   
                    Other                            13% (8%-20%)   
Upper middle income D&A                             31% (16%-47%)   
                    Other                             9% (2%-12%)   
High Income         D&A                             46% (24%-68%)   
                    Other                            12% (5%-14%)   
Global              D&A                             32% (16%-49%)   
                    Other                             9% (3%-12%)   

                                         Terrestrial ecosystems (WS>1)  \
                              population                          area   
Low income          D&A    36% (11%-54%)                 37% (25%-48%)   
                    Other  20% (18%-23%)                   7% (3%-19%)   
Lower middle income D&A    67% (37%-77%)                 64% (41%-71%)   
                    Other  1

In [112]:
ltable = table.copy()
for c in table.columns:
    ltable[c] = ltable[c].str.replace(' (',' \mbox{(',regex=False).str.replace(')',')}').str.replace('%','\%')
ltable.index = pd.MultiIndex.from_tuples(list(itertools.product(*[incs + ["Global"],["D\&A","Other"]])))
ltable.to_latex('../tables/area_pop_sums_income.tex',escape=False,multicolumn_format="L{2cm}",column_format="ll " + " ".join(["p{1cm}"]*14))

In [53]:
sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]
sector_labels = [x + " (WS>5)" if "Total" in x else x + " (WS>1)" for x in sector_labels ]

incs = [x[3:] for x in shp_ndf_df['Income category'].unique()]

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[sector_labels,["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[incs + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[sub_ndf[f"total_study_prop{suffix}"]>=threshold][var].sum()/ndf_sum)
        
    return f"{results[0]:.0%} ({results[1]:.0%}-{results[2]:.0%})"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(incs+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        group = shp_ndf_df[shp_ndf_df['Income category'].str.contains(continent)]
        ndf_ids = group.ndf_id
        doc_ids = df_ndf[df_ndf['ndf_id'].isin(ndf_ids)].doc_id
        cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
            thresh = 20
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
            thresh = 4
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector_labels[k], var)] = ci_area_sum(ndf, sub_ndf,thresh, var)       

table.to_html('../supplementary_data/area_pop_sums_income_high.md')
table.to_excel('../supplementary_data/area_pop_sums_income_high.xlsx')
                
table

Coastal and marine Ecosystems (WS>1)                 \
                                                          area     population   
Low income          D&A                             1% (0%-2%)    4% (0%-10%)   
                    Other                           0% (0%-1%)   11% (0%-11%)   
Lower middle income D&A                            3% (1%-13%)   11% (2%-31%)   
                    Other                           0% (0%-3%)     2% (2%-9%)   
Upper middle income D&A                             3% (1%-6%)   13% (4%-20%)   
                    Other                           0% (0%-1%)     0% (0%-2%)   
High Income         D&A                           11% (4%-28%)  50% (30%-73%)   
                    Other                           2% (1%-7%)     4% (3%-8%)   
Global              D&A                            5% (2%-12%)   16% (6%-29%)   
                    Other                           1% (0%-3%)     3% (1%-6%)   

                          Human and managed systems (WS>1)                 \
                                                      area     population   
Low income          D&A                       20% (5%-29%)  36% (18%-47%)   
                    Other                       4% (2%-4%)  20% (18%-20%)   
Lower middle income D&A                      31% (13%-40%)  64% (31%-74%)   
                    Other                      6% (3%-11%)  17% (14%-18%)   
Upper middle income D&A                        9% (5%-24%)  46% (32%-72%)   
                    Other                       2% (0%-5%)     3% (2%-5%)   
High Income         D&A                      17% (10%-38%)  72% (55%-83%)   
                    Other                       2% (1%-7%)     5% (3%-8%)   
Global              D&A                       14% (6%-28%)  50% (29%-66%)   
                    Other                       2% (1%-5%)    9% (8%-11%)   

                          Mountains, snow and ice (WS>1)                \
                                                    area    population   
Low income          D&A                       2% (0%-4%)    2% (0%-5%)   
                    Other                     1% (0%-1%)   5% (2%-16%)   
Lower middle income D&A                       3% (1%-6%)   5% (0%-13%)   
                    Other                     2% (1%-4%)    5% (3%-7%)   
Upper middle income D&A                       3% (1%-9%)    3% (2%-9%)   
                    Other                     0% (0%-2%)    0% (0%-1%)   
High Income         D&A                      8% (2%-22%)  25% (8%-55%)   
                    Other                     2% (1%-3%)    4% (2%-6%)   
Global              D&A                      4% (1%-10%)   7% (2%-15%)   
                    Other                     1% (0%-2%)    3% (1%-5%)   

                          Rivers, lakes, and soil moisture (WS>1)  \
                                                             area   
Low income          D&A                               4% (0%-15%)   
                    Other                              2% (2%-4%)   
Lower middle income D&A                               6% (4%-30%)   
                    Other                              7% (3%-8%)   
Upper middle income D&A                               8% (3%-22%)   
                    Other                              1% (0%-3%)   
High Income         D&A                              17% (5%-36%)   
                    Other                              4% (1%-9%)   
Global              D&A                               9% (3%-23%)   
                    Other                              3% (1%-5%)   

                                         Terrestrial ecosystems (WS>1)  \
                              population                          area   
Low income          D&A      8% (0%-30%)                  13% (1%-32%)   
                    Other  16% (16%-20%)                    2% (2%-5%)   
Lower middle income D&A     15% (6%-57%)                  22% (7%-51%)   
                    Other   17% (9%-17%)

In [ ]:
ltable = table.copy()
for c in table.columns:
    ltable[c] = ltable[c].str.replace(' (',' \mbox{(',regex=False).str.replace(')',')}').str.replace('%','\%')
ltable.index = pd.MultiIndex.from_tuples(list(itertools.product(*[incs + ["Global"],["D\&A","Other"]])))
ltable.to_latex('../tables/area_pop_sums_income_high.tex',escape=False,multicolumn_format="L{2cm}",column_format="ll " + " ".join(["p{1cm}"]*14))

In [62]:
import numpy as np

sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]

sector_total = [" - Total - "," - Total - "," - Total - "]
sector_labels = ["Low","Robust","High"]
threshs = [0,5,20]
mthreshs = [5,20,np.inf]

incs = [x[3:] for x in shp_ndf_df['Income category'].unique()]

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[sector_labels,["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[incs + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, mthreshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[(sub_ndf[f"total_study_prop{suffix}"]>=threshold) & (sub_ndf[f"total_study_prop{suffix}"]<mthreshold)][var].sum()/ndf_sum)
        
    return f"{results[0]:.0%} ({results[1]:.0%}-{results[2]:.0%})"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(incs+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        group = shp_ndf_df[shp_ndf_df['Income category'].str.contains(continent)]
        ndf_ids = group.ndf_id
        doc_ids = df_ndf[df_ndf['ndf_id'].isin(ndf_ids)].doc_id
        cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
        thresh = threshs[k]
        mthreshold = mthreshs[k]
                               
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector_labels[k], var)] = ci_area_sum(ndf, sub_ndf, thresh, mthreshold, var)       

table.to_html('../supplementary_data/area_pop_sums_evidence_types_income_high.md')
table.to_excel('../supplementary_data/area_pop_sums_evidence_types_income_high.xlsx')
                
table

Low                        Robust  \
                                    area     population           area   
Low income          D&A    40% (47%-30%)  23% (30%-14%)  26% (25%-20%)   
                    Other  21% (23%-19%)     6% (8%-4%)     3% (1%-4%)   
Lower middle income D&A    20% (38%-13%)    5% (11%-2%)  32% (32%-31%)   
                    Other    6% (13%-2%)     0% (1%-0%)    9% (4%-12%)   
Upper middle income D&A    40% (52%-32%)    9% (17%-5%)  30% (26%-26%)   
                    Other    5% (10%-3%)     0% (1%-0%)     8% (4%-7%)   
High Income         D&A     11% (19%-4%)     3% (6%-2%)  36% (39%-19%)   
                    Other    9% (10%-6%)     0% (0%-0%)     8% (9%-7%)   
Global              D&A    32% (43%-25%)   12% (19%-7%)  30% (28%-24%)   
                    Other    9% (13%-7%)     1% (2%-1%)     7% (4%-7%)   

                                                   High                 
                              population           area     population  
Low income          D&A    33% (33%-20%)    8% (2%-23%)  17% (10%-39%)  
                    Other     2% (0%-3%)     2% (2%-4%)  18% (18%-20%)  
Lower middle income D&A    16% (52%-12%)   25% (8%-34%)  61% (19%-68%)  
                    Other     1% (2%-1%)     7% (5%-9%)  17% (16%-18%)  
Upper middle income D&A    34% (48%-18%)   15% (7%-27%)  50% (29%-71%)  
                    Other     1% (3%-1%)     2% (1%-5%)     5% (2%-5%)  
High Income         D&A      7% (13%-4%)  32% (21%-55%)  81% (72%-85%)  
                    Other     1% (3%-0%)     5% (3%-9%)     8% (6%-9%)  
Global              D&A    26% (41%-16%)   18% (9%-32%)  48% (26%-63%)  
                    Other     1% (2%-1%)     3% (2%-6%)   10% (9%-11%)

In [24]:
sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]
sector_total = [" - Total - "]
sector_labels = ["Gridcell sums"]

incs = [x[3:] for x in shp_ndf_df['Income category'].unique()]

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[sector_labels,["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[incs + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[sub_ndf[f"total_study_prop{suffix}"]>=threshold][var].sum()/ndf_sum)
        
    if threshold==0:
        return f"{results[0]:.0%}"
    else:
        return f"{results[0]:.0%} ({results[1]:.0%}-{results[2]:.0%})"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(incs+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        group = shp_ndf_df[shp_ndf_df['Income category'].str.contains(continent)]
        ndf_ids = group.ndf_id
        doc_ids = df_ndf[df_ndf['ndf_id'].isin(ndf_ids)].doc_id
        cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
            thresh = 5
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
            thresh = 1
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector_labels[k], var)] = ci_area_sum(ndf, sub_ndf,0, var)       

table.to_html('../supplementary_data/area_pop_sums_income_all.md')
table.to_excel('../supplementary_data/area_pop_sums_income_all.xlsx')
                
table

Gridcell sums           
                                   area population
Low income          D&A             74%        73%
                    Other           26%        27%
Lower middle income D&A             78%        81%
                    Other           22%        19%
Upper middle income D&A             85%        94%
                    Other           15%         6%
High Income         D&A             78%        91%
                    Other           22%         9%
Global              D&A             80%        87%
                    Other           20%        13%

In [20]:
ltable = table.copy()
for c in table.columns:
    ltable[c] = ltable[c].str.replace(' (',' \mbox{(',regex=False).str.replace(')',')}').str.replace('%','\%')
ltable.index = pd.MultiIndex.from_tuples(list(itertools.product(*[incs + ["Global"],["D\&A","Other"]])))
ltable.to_latex('../tables/area_pop_sums_income_all.tex',escape=False,multicolumn_format="L{2cm}",column_format="ll " + " ".join(["p{1cm}"]*14))

In [21]:
sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]
sector_total = [" - Total - "]
sector_labels = ["Gridcell sums"]

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[sector_labels,["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[sub_ndf[f"total_study_prop{suffix}"]>=threshold][var].sum()/ndf_sum)
        
    return f"{results[0]:.0%}"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(continents+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        ndf_ids = shp_ndf_df[shp_ndf_df['CONTINENT']==continent].ndf_id
        group = doc_continents[doc_continents["continent"]==continent] 
        cdf = impact_drivers[impact_drivers['id'].isin(group.doc_id)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
            thresh = 5
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
            thresh = 1
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector_labels[k], var)] = ci_area_sum(ndf, sub_ndf,0, var)       

table.to_html('../supplementary_data/area_pop_sums_all.md')
table.to_excel('../supplementary_data/area_pop_sums_all.xlsx')
                
table

Gridcell sums           
                             area population
South America D&A             64%        81%
              Other           36%        19%
North America D&A             71%        72%
              Other           29%        28%
Africa        D&A             73%        89%
              Other           27%        11%
Europe        D&A             95%       100%
              Other            5%         0%
Asia          D&A             93%        85%
              Other            7%        15%
Oceania       D&A             86%        95%
              Other           14%         5%
Global        D&A             80%        87%
              Other           20%        13%

In [22]:
ltable = table.copy()
for c in table.columns:
    ltable[c] = ltable[c].str.replace(' (',' \mbox{(',regex=False).str.replace(')',')}').str.replace('%','\%')
ltable.index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global"],["D\&A","Other"]])))
ltable.to_latex('../tables/area_pop_sums_all.tex',escape=False,multicolumn_format="L{2cm}",column_format="ll " + " ".join(["p{1cm}"]*14))

In [29]:
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global","Without location"],["D&A","Other"]])))

sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]
table = pd.DataFrame(index=glob_index, columns=sector_labels)

df_studies = pd.read_csv('../data/study_da.csv')

da_studies = df_studies[df_studies[['precip_da_prop','temp_da_prop']].max(axis=1)>0]

def ci(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return f"{mid_df.shape[0]} ({lower_df.shape[0]}-{upper_df.shape[0]})"

for i, continent in enumerate(continents+["Global","Without location"]):
    if continent=="Global":
        doc_ids = df_ndf.doc_id.unique()
    elif continent=="Without location":
        doc_ids = set(impact_drivers.id) - set(df_ndf.doc_id.unique())
    else:
        doc_ids = doc_continents[doc_continents["continent"]==continent].doc_id
    cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    dadfs = [cdf[cdf['id'].isin(da_studies.id)],cdf[~cdf['id'].isin(da_studies.id)]]
    for j, da in enumerate(["D&A","Other"]):
        cdf = dadfs[j]
        for k, sector in enumerate(sector_total):
            if "Total" in sector:
                table.loc[(continent,da),sector_labels[k]] = ci(cdf,['0 - relevance - mean_prediction'])
            else:
                table.loc[(continent,da),sector_labels[k]] = ci(cdf,['0 - relevance - mean_prediction',sector])
            
table.to_html('../supplementary_data/continent_sector_n_studies.md')
table.to_excel('../supplementary_data/continent_sector_n_studies.xlsx')

for c in table.columns:
    table[c] = table[c].str.replace('(','\mbox{(').str.replace(')',')}')
table.index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global","Without location"],["D\&A","Other"]])))
table.to_latex('../tables/continent_sector_n_studies.tex',column_format="ll " + " ".join(["p{1.5cm}" for x in table.columns]),escape=False)
#table.to_latex('../tables/continent_sector_n_studies.tex')
table

Coastal and marine Ecosystems  \
South America    D\&A           365 \mbox{(194-662)}   
                 Other           209 \mbox{(74-520)}   
North America    D\&A        2429 \mbox{(1304-4241)}   
                 Other        1620 \mbox{(579-3859)}   
Africa           D\&A           448 \mbox{(219-881)}   
                 Other          345 \mbox{(105-865)}   
Europe           D\&A         1300 \mbox{(628-2426)}   
                 Other         678 \mbox{(222-1900)}   
Asia             D\&A         1164 \mbox{(604-2097)}   
                 Other         712 \mbox{(256-1856)}   
Oceania          D\&A          890 \mbox{(499-1484)}   
                 Other         594 \mbox{(235-1239)}   
Global           D\&A        5529 \mbox{(2957-9755)}   
                 Other       3465 \mbox{(1277-8215)}   
Without location D\&A                 0 \mbox{(0-0)}   
                 Other       3613 \mbox{(1561-7413)}   

                       Human and managed systems  Mountains, snow and ice  \
South America    D\&A       460 \mbox{(275-925)}     296 \mbox{(184-491)}   
                 Other       130 \mbox{(61-322)}       57 \mbox{(35-116)}   
North America    D\&A     1708 \mbox{(936-3300)}   1734 \mbox{(972-3203)}   
                 Other     446 \mbox{(180-1240)}    608 \mbox{(295-1259)}   
Africa           D\&A     1102 \mbox{(625-2039)}     268 \mbox{(143-514)}   
                 Other      364 \mbox{(164-956)}       79 \mbox{(44-199)}   
Europe           D\&A     1637 \mbox{(934-3162)}    985 \mbox{(547-1937)}   
                 Other       162 \mbox{(70-620)}     344 \mbox{(198-723)}   
Asia             D\&A    3516 \mbox{(2124-6329)}   1105 \mbox{(700-2011)}   
                 Other     917 \mbox{(410-2383)}     275 \mbox{(144-531)}   
Oceania          D\&A       494 \mbox{(271-979)}     349 \mbox{(191-622)}   
                 Other       117 \mbox{(50-367)}       79 \mbox{(44-192)}   
Global           D\&A   7729 \mbox{(4487-14423)}  3783 \mbox{(2201-7050)}   
                 Other    1870 \mbox{(823-5013)}   1423 \mbox{(750-2912)}   
Without location D\&A             0 \mbox{(0-0)}           0 \mbox{(0-0)}   
                 Other   2587 \mbox{(1213-6704)}   1101 \mbox{(575-2266)}   

                       Rivers, lakes, and soil moisture  \
South America    D\&A              384 \mbox{(207-794)}   
                 Other             234 \mbox{(141-469)}   
North America    D\&A           2621 \mbox{(1415-4899)}   
                 Other           1715 \mbox{(876-3595)}   
Africa           D\&A             747 \mbox{(393-1422)}   
                 Other            531 \mbox{(259-1077)}   
Europe           D\&A            1307 \mbox{(743-2638)}   
                 Other            834 \mbox{(439-1850)}   
Asia             D\&A           2968 \mbox{(1790-5292)}   
                 Other          1987 \mbox{(1138-3699)}   
Oceania          D\&A              521 \mbox{(296-993)}   
                 Other             272 \mbox{(125-568)}   
Global           D\&A          6623 \mbox{(3777-12384)}   
                 Other          4416 \mbox{(2391-8910)}   
Without location D\&A                    0 \mbox{(0-0)}   
                 Other           1766 \mbox{(887-3867)}   

                            Terrestrial ecosystems              Other systems  \
South America    D\&A       1366 \mbox{(699-2681)}      830 \mbox{(331-1744)}   
                 Other         235 \mbox{(94-592)}        205 \mbox{(56-689)}   
North America    D\&A     7835 \mbox{(4308-13552)}   5614 \mbox{(2485-11662)}   
                 Other     2259 \mbox{(1029-4821)}     2307 \mbox{(799-6068)}   
Africa           D\&A       1556 \mbox{(706-2951)}     1246 \mbox{(470-2725)}   
                 Other       447 \mbox{(182-1036)}       496 \mbox{(99-1488)}   
Europe           D\&A     6006 \mbox{(3389-10396)}    2850 \mbox{(1150-6705)}   
                 Other       816 \mbox{(361-2020)}      956 \mbox{(287-2893)}   
Asia             D\&A     5

In [44]:
glob_index = pd.Index(continents + ["Global","Without location"])

sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]
table = pd.DataFrame(index=glob_index, columns=sector_labels)

df_studies = pd.read_csv('../data/study_da.csv')

da_studies = df_studies[df_studies[['precip_da_prop','temp_da_prop']].max(axis=1)>0]

def ci(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return mid_df.shape[0]

for i, continent in enumerate(continents+["Global","Without location"]):
    if continent=="Global":
        doc_ids = df_ndf.doc_id.unique()
    elif continent=="Without location":
        doc_ids = set(impact_drivers.id) - set(df_ndf.doc_id.unique())
    else:
        doc_ids = doc_continents[doc_continents["continent"]==continent].doc_id
    cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    dadfs = [cdf[cdf['id'].isin(da_studies.id)],cdf[~cdf['id'].isin(da_studies.id)]]
    for j, da in enumerate(["D&A"]):
        for k, sector in enumerate(sector_total):
            if "Total" in sector:
                table.loc[continent,sector_labels[k]] = ci(dadfs[j],['0 - relevance - mean_prediction']) / ci(cdf,['0 - relevance - mean_prediction'])
            else:
                table.loc[continent,sector_labels[k]] = ci(dadfs[j],['0 - relevance - mean_prediction',sector]) / ci(cdf,['0 - relevance - mean_prediction',sector])
            

table.sort_values('Total')

,Coastal and marine Ecosystems,Human and managed systems,"Mountains, snow and ice","Rivers, lakes, and soil moisture",Terrestrial ecosystems,Other systems,Total
Without location,0,0,0,0,0,0,0
Africa,0.564943,0.751705,0.772334,0.584507,0.776835,0.71527,0.702799
North America,0.599901,0.792943,0.740393,0.604474,0.776204,0.708749,0.710319
Global,0.614743,0.805188,0.726662,0.599964,0.822002,0.737924,0.737494
Oceania,0.59973,0.808511,0.815421,0.656999,0.812566,0.7682,0.740411
Asia,0.620469,0.793142,0.800725,0.598991,0.854543,0.769779,0.755362
South America,0.635889,0.779661,0.838527,0.621359,0.853217,0.801932,0.775924
Europe,0.65723,0.90995,0.741159,0.610462,0.880387,0.748818,0.788092


In [45]:
glob_index = pd.Index(continents + ["Global","Without location"])

sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]
table = pd.DataFrame(index=glob_index, columns=sector_labels)

df_studies = pd.read_csv('../data/study_da.csv')

da_studies = df_studies[df_studies[['precip_da_prop','temp_da_prop']].max(axis=1)>0.5]

def ci(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return mid_df.shape[0]

for i, continent in enumerate(continents+["Global","Without location"]):
    if continent=="Global":
        doc_ids = df_ndf.doc_id.unique()
    elif continent=="Without location":
        doc_ids = set(impact_drivers.id) - set(df_ndf.doc_id.unique())
    else:
        doc_ids = doc_continents[doc_continents["continent"]==continent].doc_id
    cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    dadfs = [cdf[cdf['id'].isin(da_studies.id)],cdf[~cdf['id'].isin(da_studies.id)]]
    for j, da in enumerate(["D&A"]):
        for k, sector in enumerate(sector_total):
            if "Total" in sector:
                table.loc[continent,sector_labels[k]] = ci(dadfs[j],['0 - relevance - mean_prediction']) / ci(cdf,['0 - relevance - mean_prediction'])
            else:
                table.loc[continent,sector_labels[k]] = ci(dadfs[j],['0 - relevance - mean_prediction',sector]) / ci(cdf,['0 - relevance - mean_prediction',sector])
            

table.sort_values('Total')

,Coastal and marine Ecosystems,Human and managed systems,"Mountains, snow and ice","Rivers, lakes, and soil moisture",Terrestrial ecosystems,Other systems,Total
Without location,0,0,0,0,0,0,0
Oceania,0.462264,0.713584,0.502336,0.394704,0.6151,0.531487,0.537547
North America,0.484564,0.699629,0.559778,0.439806,0.664949,0.531625,0.569921
Africa,0.462799,0.675307,0.512968,0.406103,0.662007,0.575201,0.573937
South America,0.508711,0.664407,0.521246,0.364078,0.71143,0.607729,0.6
Global,0.499444,0.723721,0.537073,0.448591,0.731725,0.59546,0.617124
Asia,0.513859,0.692082,0.678986,0.465792,0.770879,0.656332,0.647835
Europe,0.585945,0.87771,0.604214,0.504437,0.833187,0.661324,0.717569


In [38]:
glob_index = pd.Index(continents + ["Global","Without location"])

sector_total = [" - Total - "]
sector_labels = ["Gridcell sums"]
table = pd.DataFrame(index=glob_index, columns=sector_labels)

df_studies = pd.read_csv('../data/study_da.csv')

da_studies = df_studies[df_studies[['precip_da_prop','temp_da_prop']].max(axis=1)>0.5]

def ci(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return mid_df.shape[0]

for i, continent in enumerate(continents+["Global","Without location"]):
    if continent=="Global":
        doc_ids = df_ndf.doc_id.unique()
    elif continent=="Without location":
        doc_ids = set(impact_drivers.id) - set(df_ndf.doc_id.unique())
    else:
        doc_ids = doc_continents[doc_continents["continent"]==continent].doc_id
    cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    dadfs = [cdf[cdf['id'].isin(da_studies.id)],cdf[~cdf['id'].isin(da_studies.id)]]
    for j, da in enumerate(["D&A"]):
        for k, sector in enumerate(sector_total):
            if "Total" in sector:
                table.loc[continent,sector_labels[k]] = ci(dadfs[j],['0 - relevance - mean_prediction']) / ci(cdf,['0 - relevance - mean_prediction'])
            else:
                table.loc[continent,sector_labels[k]] = ci(cdf,['0 - relevance - mean_prediction',sector])
            

table.sort_values('Gridcell sums')

,Gridcell sums
Without location,0
Oceania,0.537547
North America,0.569921
Africa,0.573937
South America,0.6
Global,0.617124
Asia,0.647835
Europe,0.717569


In [19]:
glob_index = pd.Index(continents + ["Global","Without location"])

sector_total = sector_cols + [" - Total - "]
sector_labels = [x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total]

table = pd.DataFrame(index=glob_index, columns=sector_labels)

df_studies = pd.read_csv('../data/study_da.csv')

da_studies = df_studies[df_studies[['precip_da_prop','temp_da_prop']].max(axis=1)>0]

def ci(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return mid_df.shape[0]

for i, continent in enumerate(continents+["Global","Without location"]):
    if continent=="Global":
        doc_ids = df_ndf.doc_id.unique()
    elif continent=="Without location":
        doc_ids = set(impact_drivers.id) - set(df_ndf.doc_id.unique())
    else:
        doc_ids = doc_continents[doc_continents["continent"]==continent].doc_id
    cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]

    for k, sector in enumerate(sector_total):
        if "Total" in sector:
            table.loc[continent,sector_labels[k]] = ci(cdf,['0 - relevance - mean_prediction'])
        else:
            table.loc[continent,sector_labels[k]] = ci(cdf,['0 - relevance - mean_prediction',sector])
                
table

,Coastal and marine Ecosystems,Human and managed systems,"Mountains, snow and ice","Rivers, lakes, and soil moisture",Terrestrial ecosystems,Other systems,Total
South America,574,590,353,618,1601,1035,4735
North America,4049,2154,2342,4336,10094,7921,30613
Africa,793,1466,347,1278,2003,1742,7574
Europe,1978,1799,1329,2141,6822,3806,17753
Asia,1876,4433,1380,4955,6813,8418,27649
Oceania,1484,611,428,793,1894,2239,7404
Global,8994,9599,5206,11039,23871,19647,77785
Without location,3613,2587,1101,1766,11117,3956,23954


In [20]:
table

,Coastal and marine Ecosystems,Human and managed systems,"Mountains, snow and ice","Rivers, lakes, and soil moisture",Terrestrial ecosystems,Other systems,Total
South America,574,590,353,618,1601,1035,4735
North America,4049,2154,2342,4336,10094,7921,30613
Africa,793,1466,347,1278,2003,1742,7574
Europe,1978,1799,1329,2141,6822,3806,17753
Asia,1876,4433,1380,4955,6813,8418,27649
Oceania,1484,611,428,793,1894,2239,7404
Global,8994,9599,5206,11039,23871,19647,77785
Without location,3613,2587,1101,1766,11117,3956,23954


In [21]:
hshare = table["Human and managed systems"] / table["Total"]
hshare.sort_values()

North America       0.070362
Oceania             0.082523
Europe              0.101335
Without location    0.107999
Global              0.123404
South America       0.124604
Asia                0.160331
Africa              0.193557
dtype: float64

In [25]:
hshare = (table["Terrestrial ecosystems"] + table["Coastal and marine Ecosystems"]) / table["Total"]
hshare.sort_values()

Asia                0.314261
Africa              0.369158
Global              0.422511
Oceania             0.456240
South America       0.459345
North America       0.461993
Europe              0.495691
Without location    0.614929
dtype: float64

In [26]:
hshare = table["Mountains, snow and ice"] / table["Total"]
hshare.sort_values()

Africa              0.045815
Without location    0.045963
Asia                0.049911
Oceania             0.057807
Global              0.066928
South America       0.074551
Europe              0.074861
North America       0.076503
dtype: float64

In [27]:
hshare = table["Rivers, lakes, and soil moisture"] / table["Total"]
hshare.sort_values()

Without location    0.073725
Oceania             0.107104
Europe              0.120599
South America       0.130517
North America       0.141639
Global              0.141917
Africa              0.168735
Asia                0.179211
dtype: float64

In [28]:
table

,Coastal and marine Ecosystems,Human and managed systems,"Mountains, snow and ice","Rivers, lakes, and soil moisture",Terrestrial ecosystems,Other systems,Total
South America,574,590,353,618,1601,1035,4735
North America,4049,2154,2342,4336,10094,7921,30613
Africa,793,1466,347,1278,2003,1742,7574
Europe,1978,1799,1329,2141,6822,3806,17753
Asia,1876,4433,1380,4955,6813,8418,27649
Oceania,1484,611,428,793,1894,2239,7404
Global,8994,9599,5206,11039,23871,19647,77785
Without location,3613,2587,1101,1766,11117,3956,23954
